In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

import re
from datetime import *

import matplotlib.pyplot as plt
plt.style.use('ggplot')
import plotly.graph_objects as go
import plotly.express as px
# import chart_studio
# import chart_studio.plotly as py

import seaborn as sns
import time

from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Leemos los archivos

In [ ]:
df_index = pd.read_csv('../input/evol-metricas-prestadores/index_50.csv')
df_metricas = pd.read_csv('../input/evol-metricas-prestadores/metricas_prestador_plan.csv')

# df_index = pd.read_csv('data/index_50.csv')
# df_metricas = pd.read_csv('data/metricas_prestador_plan.csv')


print(df_metricas.shape)
df_metricas.sample(3)

In [ ]:
df_procedimientos = pd.read_csv('../input/evol-metricas-prestadores/procedimientos.csv')
# df_procedimientos = pd.read_csv('data/procedimientos.csv')

print(df_procedimientos.shape)
df_procedimientos.sample(3)


In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
sns.set_theme(style="whitegrid")
sns.barplot(y = 'afiliado_count', x = 'Periodo', data = df_metricas, palette="Blues_d", ax = ax).\
    set(title='Evolución mensual - Todos los prestadores')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.show()

El gráfico anterior permite visualizar claramente el impacto de la pandemia y la cuarentena implementada a mediados de marzo 2020

## Oferta de servicios de los prestadores según PLAN
NO TODOS LOS PRESTADORES OFRECEN SERVICIOS LOS 30 MESES NI A TODOS LOS PLANES

In [ ]:
aa = df_metricas.groupby(['Prestador_ID', 'Agrupador_Plan']).Periodo.count()
sns.histplot(aa.values);

No todos los prestadores atienden a todos los planes. 
Abajo una lista con la cantidad de prestadores por cada grupo de plan:

In [ ]:
df = pd.DataFrame(aa).reset_index(drop = False)
df.groupby('Agrupador_Plan')
mask_PP2 = df.Agrupador_Plan == 'PP2'
print('Cantidad de prestadores con servicios para el plan PP2: ', mask_PP2.sum(), 'de 98\n')
for i in df.Agrupador_Plan.unique():
    mask = df.Agrupador_Plan == i 
    print(i, mask.sum())

In [ ]:
df = df_metricas.copy()
df.drop(columns = ['Prestador_ID'], inplace = True)
col_numericas = [i for i in df.columns if df.loc[:,i].apply(np.isreal).all()]
col_numericas

In [ ]:
# ### Creamos la matriz de correlacion y ordenos de mayor a menor
ccc = df[col_numericas].corr()
# ccc

plt.figure(figsize=(20, 8))
# define the mask to set the values in the upper triangle to True
mask = np.triu(np.ones_like(ccc, dtype=np.bool))
heatmap = sns.heatmap(ccc, mask=mask, vmin=-0.5, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Triangle Correlation Heatmap', fontdict={'fontsize':18}, pad=16);

In [ ]:
ll = ['afiliado_nunique',
 'afiliado_count',
 'sexo_afiliado_prop_mujeres',
 'monto_prestacion_q0.50',
#  'monto_prestacion_q0.95',
#  'edad_afil_median',
 'edad_afil_q0.50',
#  'edad_afil_q0.95',
#  'permanencia_median',
 'permanencia_q0.50',
# 'Agrupador_Plan', 
#  'permanencia_q0.95'
     ]
df = df_metricas[ll].sample(100)
plt.figure(figsize=(20, 10))
pplot = sns.pairplot(df, kind="hist", corner=True);
# pplot = sns.pairplot(df);

## Inflación efectiva 

In [ ]:
plt.figure(figsize=(10,5))
g = sns.lineplot(
    data=df_index,
    x='Periodo',
    y= 'Index_50',
    palette='Set1'
)
g.set_xticklabels(labels=df_index.Periodo, rotation=90)
plt.show()

## Influencia de la EDAD en el CONSUMO

In [ ]:
## Cual es la edad de cada plan
aa = df_metricas.groupby('Agrupador_Plan').agg({'edad_afil_q0.50' : 'mean'}).reset_index(drop = False)
aa['anios'] = round(aa['edad_afil_q0.50']/12,1)
aa.sort_values(by = 'anios', ascending = False)
# df_metricas['edad_afil_q0.50']

In [ ]:
## Cuales son los prestadores q atienden a gente mayor

aa = df_metricas.groupby('Prestador_ID').agg({'edad_afil_q0.50' : 'mean'}).reset_index(drop = False)
aa['anios'] = round(aa['edad_afil_q0.50']/12,1)
aa.sort_values(by = 'anios', ascending = False)



In [ ]:
## independientemente del prestador y del plan analizamos el consumo vs la edad

plt.figure(figsize=(10,5))
g = sns.scatterplot(
    data=df_metricas,
    x='edad_afil_q0.50',
    y= 'monto_prestacion_q0.50',
    hue = 'Agrupador_Plan',
    palette='Set1'
)
# g.set_xticklabels(labels=df_index.Periodo, rotation=90)
plt.show()

## Afiliados que están hace más tiempo en la OS gastan más??